```{autolink-concat}
```

# K-matrix

```{autolink-concat}

```

<!-- cspell:ignore amma -->

While {mod}`ampform` does not yet provide a generic way to formulate an amplitude model with $\boldsymbol{K}$-matrix dynamics, the (experimental) {mod}`.kmatrix` module makes it fairly simple to produce a symbolic expression for a parameterized $\boldsymbol{K}$-matrix with an arbitrary number of poles and channels and play around with it interactively. For more info on the $\boldsymbol{K}$-matrix, see the classic paper by Chung {cite}`Chung:1995dx`, {pdg-review}`2021; Resonances`, or this instructive presentation {cite}`Meyer:2008-MatrixTutorial`.

Section {ref}`usage/dynamics/k-matrix:Physics` summarizes {cite}`Chung:1995dx`, so that the {mod}`.kmatrix` module can reference to the equations. It also points out some subtleties and deviations.

:::{note}

The $\boldsymbol{K}$-matrix approach was originally worked worked out in {doc}`compwa-report:005/index`, {doc}`compwa-report:009/index`, and {doc}`compwa-report:010/index`. Those reports contained a few mistakes, which have been addressed here.

:::

```{autolink-skip}
```

In [ ]:
%matplotlib widget

In [ ]:
import inspect
import logging
import os
import re
import warnings

import graphviz
import ipywidgets as w
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ipympl.backend_nbagg import Canvas
from IPython.display import Image, display
from matplotlib import cm
from matplotlib.transforms import Bbox

from ampform.dynamics import kmatrix, phasespace, relativistic_breit_wigner
from ampform.sympy import partial_doit, rename_symbols
from ampform.sympy.slider import create_slider, substitute_indexed_symbols


def hide_toolbar(canvas: Canvas) -> None:
    canvas.toolbar_visible = False
    canvas.header_visible = False
    canvas.footer_visible = False


STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)
logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

In [ ]:
type(plt.subplots()[0].canvas)

## Physics

The $\boldsymbol{K}$-matrix formalism is used to describe coupled, two-body **formation processes** of the form $c_j d_j \to R \to a_i b_i$, with $i,j$ representing each separate channel and $R$ an intermediate state by which these channels are coupled.

In [ ]:
dot = """
digraph {
    rankdir=LR;
    node [shape=point, width=0];
    edge [arrowhead=none];
    "Na" [shape=none, label="aᵢ"];
    "Nb" [shape=none, label="bᵢ"];
    "Nc" [shape=none, label="cⱼ"];
    "Nd" [shape=none, label="dⱼ"];
    { rank=same "Nc", "Nd" };
    { rank=same "Na", "Nb" };
    "Nc" -> "N0";
    "Nd" -> "N0";
    "N1" -> "Na";
    "N1" -> "Nb";
    "N0" -> "N1" [label="R"];
    "N0" [shape=none, label=""];
    "N1" [shape=none, label=""];
}
"""
graphviz.Source(dot)

A small adaptation allows us to describe a coupled, two-body **production process** of the form $R \to a_ib_i$ (see {ref}`usage/dynamics/k-matrix:Production processes`).

In the following, $n$ denotes the number of channels and $n_R$ the number of poles. In the {mod}`.kmatrix` module, we use $0 \leq i,j < n$ and $1 \leq R \leq n_R$.

### Partial wave expansion

In amplitude analysis, the main aim is to express the differential cross section $\frac{d\sigma}{d\Omega}$, that is, the intensity distribution in each spherical direction $\Omega=(\phi,\theta)$ as we can observe in experiments. This differential cross section can be expressed in terms of the **scattering amplitude** $A$:

```{margin}
{cite}`Chung:1995dx` Eq. (1)
```

$$
\frac{d\sigma}{d\Omega} = \left|A(\Omega)\right|^2.
$$ (differential cross section)

We can now further express $A$ in terms of **partial wave amplitudes** by expanding it in terms of its angular momentum components:[^spin-formalisms]

```{margin}
{cite}`Chung:1995dx` Eq. (2)
```


$$

A(\Omega) = \frac{1}{q*i}\sum_L\left(2L+1\right) T_L(s) {D^{\*L}*{\lambda\mu}}\left(\phi,\theta,0\right)

$$
(partial-wave-expansion)

with $L$ the total angular momentum of the decay product pair, $\lambda=\lambda_a-\lambda_b$ and $\mu=\lambda_c-\lambda_d$ the helicity differences of the final and initial states, $D$ a [Wigner-$D$ function](https://en.wikipedia.org/wiki/Wigner_D-matrix), and $T_J$ an operator representing the partial wave amplitude.

[^spin-formalisms]: Further subtleties arise when taking spin into account, especially for sequential decays. This is where {doc}`spin formalisms </usage/helicity/formalism>` come in.

The above sketch is just with one channel in mind. The same holds true though for a number of channels $n$, with the only difference being that the $T$ operator becomes an $n\times n$ $\boldsymbol{T}$-matrix.
$$

### Transition operator

The important point is that we have now expressed $A$ in terms of an angular part (depending on $\Omega$) and a dynamical part $\boldsymbol{T}$ that depends on the [Mandelstam variable](https://pwa.rtfd.io/physics/#mandelstam-variables) $s$.

The dynamical part $\boldsymbol{T}$ is usually called the **transition operator**. It describes the interacting part of the more general **scattering operator** $\boldsymbol{S}$, which describes the (complex) amplitude $\langle f|\boldsymbol{S}|i\rangle$ of an initial state $|i\rangle$ transitioning to a final state $|f\rangle$. The scattering operator describes both the non-interacting amplitude and the transition amplitude, so it relates to the transition operator as:

```{margin}
{cite}`Chung:1995dx` Eq. (10)
```

$$
\boldsymbol{S} = \boldsymbol{I} + 2i\boldsymbol{T}
$$ (S in terms of T)

with $\boldsymbol{I}$ the identity operator. Just like in {cite}`Chung:1995dx`, we use a factor 2, while other authors choose $\boldsymbol{S} = \boldsymbol{I} + i\boldsymbol{T}$. In that case, one would have to multiply Eq. {eq}`partial-wave-expansion` by a factor $\frac{1}{2}$.
$$

### Ensuring unitarity

Knowing the origin of the $\boldsymbol{T}$-matrix, there is an important restriction that we need to comply with when we further formulate a {ref}`parametrization <usage/dynamics/k-matrix:Pole parametrization>`: **unitarity**. This means that $\boldsymbol{S}$ should conserve probability, namely $\boldsymbol{S}^\dagger\boldsymbol{S} = \boldsymbol{I}$. Luckily, there is a trick that makes this easier. If we express $\boldsymbol{S}$ in terms of an operator $\boldsymbol{K}$ by applying a [Cayley transformation](https://en.wikipedia.org/wiki/Cayley_transform):

```{margin}
{cite}`Chung:1995dx` Eq. (20)
```

$$
\boldsymbol{S} = (\boldsymbol{I} + i\boldsymbol{K})(I - i\boldsymbol{K})^{-1},
$$ (Cayley transformation)

_unitarity is conserved if $\boldsymbol{K}$ is real_. With some matrix jumbling, we can derive that the $\boldsymbol{T}$-matrix can be expressed in terms of $\boldsymbol{K}$ as follows:

```{margin}
{cite}`Chung:1995dx` Eq. (19);
compare with {eq}`T-hat-in-terms-of-K-hat`
```


$$

\boldsymbol{T}
= \boldsymbol{K} \left(\boldsymbol{I} - i\boldsymbol{K}\right)^{-1}
= \left(\boldsymbol{I} - i\boldsymbol{K}\right)^{-1} \boldsymbol{K}.

$$
(T-in-terms-of-K)
$$

### Lorentz-invariance

The description so far did not take Lorentz-invariance into account. For this, we first need to define a **two-body phase space matrix** $\boldsymbol{\rho}$:

```{margin}
{cite}`Chung:1995dx` Eq. (36)
```

$$
\boldsymbol{\rho} = \begin{pmatrix}
\rho_0 & \cdots & 0      \\
\vdots & \ddots & \vdots \\
0      & \cdots & \rho_{n-1}
\end{pmatrix}.
$$ (rho matrix)

with $\rho_i$ given by {eq}`PhaseSpaceFactor` in {class}`.PhaseSpaceFactor` for the final state masses $m_{a,i}, m_{b,i}$. The **Lorentz-invariant amplitude $\boldsymbol{\hat{T}}$** and corresponding Lorentz-invariant $\boldsymbol{\hat{K}}$-matrix can then be computed from $\boldsymbol{T}$ and $\boldsymbol{K}$ with:[^rho-dagger]

```{margin}
{cite}`Chung:1995dx` Eqs. (34) and (47)
```


$$

\begin{eqnarray}
\boldsymbol{T} & = & \sqrt{\boldsymbol{\rho}} \; \boldsymbol{\hat{T}} \sqrt{\boldsymbol{\rho}} \\
\boldsymbol{K} & = & \sqrt{\boldsymbol{\rho}} \; \boldsymbol{\hat{K}} \sqrt{\boldsymbol{\rho}}.
\end{eqnarray}

$$
(K-hat-and-T-hat)

[^rho-dagger]: An unpublished primer on the $\boldsymbol{K}$-matrix by Chung {cite}`Chung:1995-PrimerKmatrixFormalism` uses a conjugate transpose of $\boldsymbol{\rho}$, e.g. $\boldsymbol{T} = \sqrt{\boldsymbol{\rho^\dagger}} \; \boldsymbol{\hat{T}} \sqrt{\boldsymbol{\rho}}$. This should not matter above threshold, where the phase space factor is real, but could have effects below threshold. This is where things become tricky: on the one hand, we need to ensure that $\boldsymbol{K}$ remains real (unitarity) and on the other, we need to take keep track of which imaginary square root we choose (**Riemann sheet**). The latter is often called the requirement of **analyticity**. This is currently being explored in {doc}`compwa-report:003/index` and {doc}`compwa-report:004/index`.

With these definitions, we can deduce that:

```{margin}
{cite}`Chung:1995dx` Eq. (51);
compare with {eq}`T-in-terms-of-K`
```


$$

\boldsymbol{\hat{T}}
= \boldsymbol{\hat{K}} (\boldsymbol{I} - i\boldsymbol{\rho}\boldsymbol{\hat{K}})^{-1}
= (\boldsymbol{I} - i\boldsymbol{\hat{K}}\boldsymbol{\rho})^{-1} \boldsymbol{\hat{K}}.

$$
(T-hat-in-terms-of-K-hat)
$$

### Production processes

{ref}`As noted in the intro <usage/dynamics/k-matrix:Physics>`, the $\boldsymbol{K}$-matrix describes scattering processes of type $cd \to ab$. It can however be generalized to describe **production processes** of type $R \to ab$. Here, the amplitude is described by a **final state $F$-vector** of size $n$, so the question is how to express $F$ in terms of transition matrix $\boldsymbol{T}$.

In [ ]:
dot = """
digraph {
    rankdir=LR;
    node [shape=point, width=0];
    edge [arrowhead=none];
    "a" [shape=none, label="aᵢ"];
    "b" [shape=none, label="bᵢ"];
    "R" [shape=none, label="R"];
    "N" [shape=none, label=""];
    "R" -> "N";
    "N" -> "a";
    "N" -> "b";
    { rank=same "Na", "Nb" };
}
"""
graphviz.Source(dot)

One approach by {cite}`Aitchison:1972ay` is to transform $\boldsymbol{T}$ into $F$ (and its relativistic form $\hat{F}$) through the **production amplitude $P$-vector**:

```{margin}
{cite}`Chung:1995dx` Eqs. (114) and (115)
```

$$
\begin{eqnarray}
F & = & \left(\boldsymbol{I}-i\boldsymbol{K}\right)^{-1}P \\
\hat{F} & = & \left(\boldsymbol{I}-i\boldsymbol{\hat{K}\boldsymbol{\rho}}\right)^{-1}\hat{P},
\end{eqnarray}
$$ (F-in-terms-of-P)

where we can compute $\boldsymbol{\hat{K}}$ from {eq}`K-hat-and-T-hat`:


$$

\hat{\boldsymbol{K}} = \sqrt{\boldsymbol{\rho}^{-1}} \boldsymbol{K} \sqrt{\boldsymbol{\rho}^{-1}}.

$$
(K-hat in terms of K)

Another approach by {cite}`Cahn:1985wu` further approximates this by defining a **$Q$-vector**:

```{margin}
{cite}`Chung:1995dx` Eq. (124)
```


$$

Q = \boldsymbol{K}^{-1}P \quad \mathrm{and} \quad
\hat{Q} = \boldsymbol{\hat{K}}^{-1}\hat{P}

$$
(Q-vector)

that _is taken to be constant_ (just some 'fitting' parameters). The $F$-vector can then be expressed as:

```{margin}
{cite}`Chung:1995dx` Eq. (125)
```


$$

F = \boldsymbol{T}Q
\quad \mathrm{and} \quad
\hat{F} = \boldsymbol{\hat{T}}\hat{Q}

$$
(F in terms of Q)

Note that for all these vectors, we have:

```{margin}
{cite}`Chung:1995dx` Eqs. (116) and (124)
```


$$

F=\sqrt{\boldsymbol{\rho}}\hat{F},\quad
P=\sqrt{\boldsymbol{\rho}}\hat{P},\quad\mathrm{and}\quad
Q=\sqrt{\boldsymbol{\rho}^{-1}}\hat{Q}.

$$
(invariant-vectors)
$$

### Pole parametrization

After all these matrix definitions, the final challenge is to choose a correct parametrization for the elements of $\boldsymbol{K}$ and $P$ that accurately describes the resonances we observe.[^pole-vs-resonance] There are several choices, but a common one is the following summation over the **poles** $R$:[^complex-conjugate-parametrization]

[^complex-conjugate-parametrization]: Eqs. (51) and (52) in {cite}`Chung:1995-PrimerKmatrixFormalism` take a complex conjugate of one of the residue functions and one of the phase space factors.

```{margin}
{cite}`Chung:1995dx` Eqs. (73) and (74)
```

$$
\begin{eqnarray}
K_{ij} &=& \sum_R\frac{g_{R,i}g_{R,j}}{m_R^2-s} + c_{ij} \\
\hat{K}_{ij} &=& \sum_R \frac{g_{R,i}(s)g_{R,j}(s)}{\left(m_R^2-s\right)\sqrt{\rho_i\rho_j}} + \hat{c}_{ij}
\end{eqnarray}
$$ (K-matrix parametrization)

with $c_{ij}, \hat{c}_{ij}$ some optional background characterization and $g_{R,i}$ the **residue functions**. The residue functions are often further expressed as:

```{margin}
{cite}`Chung:1995dx` Eqs. (75-78)
```


$$

\begin{eqnarray}
g*{R,i} &=& \gamma*{R,i}\sqrt{m*R\Gamma^0*{R,i}} \\
g*{R,i}(s) &=& \gamma*{R,i}\sqrt{m*R\Gamma*{R,i}(s)}
\end{eqnarray}

$$
(residue-function)

with $\gamma_{R,i}$ some _real_ constants and $\Gamma^0_{R,i}$ the **partial width** of each pole. In the Lorentz-invariant form, the fixed width $\Gamma^0$ is replaced by an "energy dependent" {class}`.EnergyDependentWidth` $\Gamma(s)$.[^phase-space-factor-normalization] The **width** for each pole can be computed as $\Gamma^0_R = \sum_i\Gamma^0_{R,i}$.

[^phase-space-factor-normalization]: Unlike Eq. (77) in {cite}`Chung:1995dx`, AmpForm defines {class}`.EnergyDependentWidth` as in {pdg-review}`2021; Resonances; p.6`, Eq. (50.28). The difference is that the phase space factor denoted by $\rho_i$ in Eq. (77) in {cite}`Chung:1995dx` is divided by the phase space factor at the pole position $m_R$. So in AmpForm, the choice is $\rho_i \to \frac{\rho_i(s)}{\rho_i(m_R)}$.
$$

The production vector $P$ is commonly parameterized as:[^damping-factor-P-parametrization]

```{margin}
{cite}`Chung:1995dx` Eqs. (118-119) and (122)
```

$$
\begin{eqnarray}
P_i &=& \sum_R \frac{\beta^0_R\,g_{R,i}(s)}{m_R^2-s} \\
\hat{P}_i
&=& \sum_R \frac{\beta^0_R\,g_{R,i}(s)}{\left(m_R^2-s\right)\sqrt{\rho_i}} \\
&=& \sum_R \frac{
    \beta^0_R\gamma_{R,i}m_R\Gamma^0_R B_{R,i}(q(s))
}{m_R^2-s}
\end{eqnarray}
$$ (P-vector parametrization)

with $B_{R,i}(q(s))$ the **centrifugal damping factor** (see {class}`.FormFactor` and {class}`.BlattWeisskopfSquared`) for channel $i$ and $\beta_R^0$ some (generally complex) constants that describe the production information of the decaying state $R$. Usually, these constants are rescaled just like the residue functions in {eq}`residue-function`:

[^damping-factor-P-parametrization]: Just as with [^phase-space-factor-normalization], we have smuggled a bit in the last equation in order to be able to reproduce Equation (50.23) in {pdg-review}`2021; Resonances; p.9` in the case $n=1,n_R=1$, on which {func}`.relativistic_breit_wigner_with_ff` is based.

```{margin}
{cite}`Chung:1995dx` Eq. (121)
```


$$

\beta^0_R = \beta_R\sqrt{m_R\Gamma^0_R}.

$$
(beta functions)
$$

## Implementation

### Non-relativistic K-matrix

A non-relativistic $\boldsymbol{K}$-matrix for an arbitrary number of channels and an arbitrary number of poles can be formulated with the {meth}`.NonRelativisticKMatrix.formulate` method:

In [ ]:
n_poles = sp.Symbol("n_R", integer=True, positive=True)
k_matrix_nr = kmatrix.NonRelativisticKMatrix.formulate(n_poles=n_poles, n_channels=1)
k_matrix_nr[0, 0]

Notice how the $\boldsymbol{K}$-matrix reduces to a {func}`.relativistic_breit_wigner` in the case of one channel and one pole (but for a residue constant $\gamma$):

In [ ]:
k_matrix_1r = kmatrix.NonRelativisticKMatrix.formulate(n_poles=1, n_channels=1)
k_matrix_1r[0, 0].doit().simplify()

Now let's investigate the effect of using a $\boldsymbol{K}$-matrix to describe **two poles** in one channel and see how it compares with the sum of two Breit-Wigner functions (two 'resonances'). Two Breit-Wigner 'poles' with the same parameters would look like this:

In [ ]:
s, m1, m2, Gamma1, Gamma2 = sp.symbols("s m1 m2 Gamma1 Gamma2", nonnegative=True)
bw1 = relativistic_breit_wigner(s, m1, Gamma1)
bw2 = relativistic_breit_wigner(s, m2, Gamma2)
bw = bw1 + bw2
bw

while a $\boldsymbol{K}$-matrix parametrizes the two poles as:

In [ ]:
k_matrix_2r = kmatrix.NonRelativisticKMatrix.formulate(n_poles=2, n_channels=1)
k_matrix = k_matrix_2r[0, 0].doit()

In [ ]:
# reformulate terms
*rest, denominator, nominator = k_matrix.args
term1 = nominator.args[0] * denominator * sp.Mul(*rest)
term2 = nominator.args[1] * denominator * sp.Mul(*rest)
k_matrix = term1 + term2
k_matrix

To simplify things, we can set the residue constants $\gamma$ to one. Notice how the $\boldsymbol{K}$-matrix has introduced some coupling ('interference') between the two terms.

In [ ]:
def remove_residue_constants(expression):
    expression = substitute_indexed_symbols(expression)
    residue_constants = filter(
        lambda s: re.match(r"^\\?gamma", s.name),
        expression.free_symbols,
    )
    return expression.xreplace(dict.fromkeys(residue_constants, 1))


display(
    remove_residue_constants(bw),
    remove_residue_constants(k_matrix),
)

Now, just like in {doc}`/usage/interactive`, we use the {mod}`ampform.sympy.slider` module to visualize the difference between the two expressions. The important thing is that the Argand plot on the right shows that **the $\boldsymbol{K}$-matrix conserves unitarity**.

Note that we have to call {func}`.substitute_indexed_symbols` to turn the {class}`~sympy.tensor.indexed.Indexed` instances in this {obj}`~sympy.matrices.dense.Matrix` expression into {class}`~sympy.core.symbol.Symbol`s before calling this function. We also call {func}`.rename_symbols` so that the residue $\gamma$'s get a name that does not have to be dummified by {func}`~sympy.utilities.lambdify.lambdify`.

In [ ]:
# Prepare expressions
m = sp.Symbol("m", nonnegative=True)
k_matrix = substitute_indexed_symbols(k_matrix)
rename_gammas = lambda s: re.sub(r"\\([Gg])amma_{([0-9]),0}", r"\1amma\2", s)  # noqa: E731
gamma1, gamma2 = sp.symbols("gamma1 gamma2", nonnegative=True)
bw = rename_symbols(bw, rename_gammas)
k_matrix = rename_symbols(k_matrix, rename_gammas)
bw = bw.xreplace({s: m**2})
k_matrix = k_matrix.xreplace({s: m**2})

# Prepare sliders and domain
args = (m, Gamma1, Gamma2, gamma1, gamma2, m1, m2)
np_bw_1d = sp.lambdify(args, bw.doit())
np_kmatrix = sp.lambdify(args, k_matrix.doit())

m_min, m_max = 0, 3
domain_1d = np.linspace(m_min, m_max, 200)
domain_argand = np.linspace(m_min - 2, m_max + 2, 1_000)
sliders = {str(s): create_slider(s, max=2, step=0.01, value=1) for s in args[1:]}
sliders["m1"].max = 3
sliders["m2"].max = 3
sliders["m1"].value = 1.1
sliders["m2"].value = 1.9
sliders["Gamma1"].value = 0.2
sliders["Gamma2"].value = 0.3
UI = w.VBox(list(sliders.values()))

# Create figure
fig, axes = plt.subplots(
    ncols=2,
    figsize=1.2 * np.array((8, 3.8)),
)
hide_toolbar(fig.canvas)
ax1, ax2 = axes.ravel()
m_label = "$m_{a+b}$"
ax1.set_xlabel(m_label)
ax1.set_ylabel("$|A|^2$")
ax2.set_xlabel("Re($A$)")
ax2.set_ylabel("Im($A$)")
for ax in axes.ravel():
    ax.spines["left"].set_position("zero")
    ax.spines["bottom"].set_position("zero")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

LINES_BW = None


def plot_bw_sum(**kwargs):
    global LINES_BW
    z_bw = np_bw_1d(domain_1d, **kwargs)
    z_kmatrix = np_kmatrix(domain_1d, **kwargs)
    y_bw = np.abs(z_bw) ** 2
    y_kmatrix = np.abs(z_kmatrix) ** 2
    if LINES_BW is None:
        LINES_BW = [
            ax1.plot(domain_1d, y_kmatrix, label="$K$-matrix")[0],
            ax1.plot(domain_1d, y_bw, label="Breit-Wigner")[0],
            ax1.axvline(kwargs["m1"], c="gray", ls="dotted"),
            ax1.axvline(kwargs["m2"], c="gray", ls="dotted"),
            ax2.scatter(z_kmatrix.real, z_kmatrix.imag, label="$K$-matrix", s=1),
            ax2.scatter(z_bw.real, z_bw.imag, label="Breit-Wigner", s=1),
        ]
    else:
        LINES_BW[0].set_ydata(y_kmatrix)
        LINES_BW[1].set_ydata(y_bw)
        LINES_BW[2].set_xdata([kwargs["m1"]])
        LINES_BW[3].set_xdata([kwargs["m2"]])
        LINES_BW[4].set_offsets(np.stack([z_kmatrix.real, z_kmatrix.imag]).T)
        LINES_BW[5].set_offsets(np.stack([z_bw.real, z_bw.imag]).T)
    ax1.relim()
    ax1.autoscale_view()
    ax2.ignore_existing_data_limits = True
    bbox = Bbox.union([
        LINES_BW[4].get_datalim(ax2.transData),
        LINES_BW[5].get_datalim(ax2.transData),
    ])
    ax2.update_datalim(bbox)
    ax2.autoscale_view()


# Argand plots
output = w.interactive_output(plot_bw_sum, controls=sliders)
ax2.legend(loc="upper right")
plt.show()
display(UI, output)

{{ run_interactive }}

In [ ]:
if STATIC_WEB_PAGE:
    from IPython.display import Image, display
    from matplotlib.animation import FuncAnimation, PillowWriter
    from tqdm.auto import tqdm

    def frame_generator():
        while True:
            if not forward and animated_slider.value <= start:
                return
            yield

    def animate(_):
        global forward
        if animated_slider.value >= end:
            forward = False
        if forward:
            animated_slider.value += step * animated_slider.step
        else:
            animated_slider.value -= step * animated_slider.step
        fig.canvas.draw_idle()
        pbar.set_postfix_str(f"value={animated_slider.value:g}")
        pbar.update()

    ax1.set_ylim([0, 2])
    ax2.set_xlim([-1, +1])
    ax2.set_ylim([0, 2])

    animated_slider = sliders["m1"]
    start = max(1.0, animated_slider.min)
    end = min(2.7, animated_slider.max)
    step = 3

    forward = True
    animated_slider.value = start
    pbar = tqdm(desc="Exporting animation", leave=False)
    output_path = "non-relativistic-k-matrix.gif"
    animation = FuncAnimation(
        fig,
        animate,
        cache_frame_data=False,
        frames=frame_generator(),
        repeat=False,
    )
    animation.save(output_path, writer=PillowWriter(fps=20))
    pbar.close()
    with open(output_path, "rb") as f:
        display(UI, Image(data=f.read(), format="png"))

### Relativistic K-matrix

Relativistic $\boldsymbol{K}$-matrices for an arbitrary number of channels and an arbitrary number of poles can be formulated with the {meth}`.RelativisticKMatrix.formulate` method:

In [ ]:
L = sp.Symbol("L", integer=True, negative=False)
n_poles = sp.Symbol("n_R", integer=True, positive=True)
rel_k_matrix_nr = kmatrix.RelativisticKMatrix.formulate(
    n_poles=n_poles, n_channels=1, angular_momentum=L
)
rel_k_matrix_nr[0, 0]

Again, as in {ref}`usage/dynamics/k-matrix:Non-relativistic K-matrix`, the $\boldsymbol{K}$-matrix reduces to something of a {func}`.relativistic_breit_wigner`. This time, the width has been replaced by a {class}`.EnergyDependentWidth` and some {class}`.PhaseSpaceFactor`s have been inserted that take care of the decay into two decay products:

In [ ]:
rel_k_matrix_1r = kmatrix.RelativisticKMatrix.formulate(
    n_poles=1, n_channels=1, angular_momentum=L
)
partial_doit(rel_k_matrix_1r[0, 0], sp.Sum).simplify(doit=False)

Note that another difference with {func}`.relativistic_breit_wigner_with_ff` is an additional phase space factor in the denominator. That one disappears in {ref}`usage/dynamics/k-matrix:P-vector`.

The $\boldsymbol{K}$-matrix with two poles becomes (neglecting the $\sqrt{\rho_0(s)}$):

In [ ]:
rel_k_matrix_2r = kmatrix.RelativisticKMatrix.formulate(
    n_poles=2, n_channels=1, angular_momentum=L
)
rel_k_matrix_2r = partial_doit(rel_k_matrix_2r[0, 0], sp.Sum)

In [ ]:
rel_k_matrix_2r = substitute_indexed_symbols(rel_k_matrix_2r)
s, m_a, m_b = sp.symbols("s, m_a0, m_b0", nonnegative=True)
rho = phasespace.PhaseSpaceFactor(s, m_a, m_b)
rel_k_matrix_2r = rel_k_matrix_2r.xreplace({
    sp.sqrt(rho): 1,
    sp.conjugate(sp.sqrt(rho)): 1,
})
*rest, denominator, nominator = rel_k_matrix_2r.args
term1 = nominator.args[0] * denominator * sp.Mul(*rest)
term2 = nominator.args[1] * denominator * sp.Mul(*rest)
rel_k_matrix_2r = term1 + term2
rel_k_matrix_2r

This again shows the interference introduced by the $\boldsymbol{K}$-matrix, when compared with a sum of two Breit-Wigner functions.

### P-vector

For one channel and an arbitrary number of poles $n_R$, the $F$-vector gets the following form:

In [ ]:
n_poles = sp.Symbol("n_R", integer=True, positive=True)
kmatrix.NonRelativisticPVector.formulate(n_poles=n_poles, n_channels=1)[0]

The {class}`.RelativisticPVector` looks like:

In [ ]:
kmatrix.RelativisticPVector.formulate(
    n_poles=n_poles, n_channels=1, angular_momentum=L
)[0]

As in {ref}`usage/dynamics/k-matrix:Non-relativistic K-matrix`, if we take $n_R=1$, the $F$-vector reduces to a Breit-Wigner function, but now with an additional factor $\beta$.

In [ ]:
f_vector_1r = kmatrix.NonRelativisticPVector.formulate(n_poles=1, n_channels=1)
partial_doit(f_vector_1r[0], sp.Sum)

And when we neglect the phase space factors $\sqrt{\rho_0(s)}$, the {class}`.RelativisticPVector` reduces to a {func}`.relativistic_breit_wigner_with_ff`!

In [ ]:
rel_f_vector_1r = kmatrix.RelativisticPVector.formulate(
    n_poles=1, n_channels=1, angular_momentum=L
)
rel_f_vector_1r = partial_doit(rel_f_vector_1r[0], sp.Sum)

In [ ]:
rel_f_vector_1r = substitute_indexed_symbols(rel_f_vector_1r)
s, m_a, m_b = sp.symbols("s, m_a0, m_b0", nonnegative=True)
rho = phasespace.PhaseSpaceFactor(s, m_a, m_b)
rel_f_vector_1r.xreplace({
    sp.sqrt(rho): 1,
    sp.conjugate(sp.sqrt(rho)): 1,
}).simplify(doit=False)

Note that the $F$-vector approach introduces additional $\beta$-coefficients. These can constants can be complex and can introduce phase differences form the production process.

In [ ]:
f_vector_2r = kmatrix.NonRelativisticPVector.formulate(n_poles=2, n_channels=1)
f_vector = f_vector_2r[0].doit()

In [ ]:
*rest, denominator, nominator = f_vector.args
term1 = nominator.args[0] * denominator * sp.Mul(*rest)
term2 = nominator.args[1] * denominator * sp.Mul(*rest)
f_vector = term1 + term2
f_vector

Now again let's compare the compare this with a sum of two {func}`.relativistic_breit_wigner`s, now with the two additional $\beta$-constants.

In [ ]:
beta1, beta2 = sp.symbols("beta1 beta2", nonnegative=True)
bw_with_phases = beta1 * bw1 + beta2 * bw2
display(
    bw_with_phases,
    remove_residue_constants(f_vector),
)

{{ run_interactive }}

In [ ]:
# Prepare expressions
f_vector = substitute_indexed_symbols(f_vector)
rename_gammas = lambda s: re.sub(  # noqa: E731
    r"\\([Gg])amma_{([0-9]),0}", r"\1amma\2", s
)
c1, c2, phi1, phi2 = sp.symbols("c1 c2 phi1 phi2", real=True)
bw_with_phases = rename_symbols(bw_with_phases, rename_gammas)
f_vector = rename_symbols(f_vector, rename_gammas)
substitutions = {
    s: m**2,
    beta1: c1 * sp.exp(sp.I * phi1),
    beta2: c2 * sp.exp(sp.I * phi2),
}
bw_with_phases = bw_with_phases.xreplace(substitutions)
f_vector = f_vector.xreplace(substitutions)

# Prepare sliders and domain
args = (m, m1, m2, Gamma1, Gamma2, gamma1, gamma2, c1, c2, phi1, phi2)
np_f_vector = sp.lambdify(args, f_vector.doit())
np_bw = sp.lambdify(args, bw_with_phases.doit())

# Set plot domain
x_min, x_max = 0, 3
y_min, y_max = -0.5, +0.5
plot_domain = np.linspace(x_min, x_max, num=200)
plot_domain_argand = np.linspace(x_min - 2, x_max + 2, num=400)
X, Y = np.meshgrid(
    np.linspace(x_min, x_max, num=160),
    np.linspace(y_min, y_max, num=80),
)
plot_domain_complex = X + Y * 1j

# Set slider values and ranges
sliders = {str(s): create_slider(s, max=2, step=0.01) for s in args[1:]}
sliders["m1"].max = 3
sliders["m2"].max = 3
sliders["m1"].value = 1.4
sliders["m2"].value = 1.7
for key in ("phi1", "phi2"):
    sliders[key].max = np.pi
    sliders[key].step = np.pi / 20
for key in ("gamma1", "gamma2"):
    sliders[key].min = -1
    sliders[key].max = +1
sliders["c1"].value = 1
sliders["c2"].value = 1
sliders["Gamma1"].value = 0.2
sliders["Gamma2"].value = 0.3
sliders["gamma1"].value = 1 / np.sqrt(2)
sliders["gamma2"].value = 1 / np.sqrt(2)

# Create figure
fig, axes = plt.subplots(
    figsize=(10, 7),
    gridspec_kw=dict(
        bottom=0.07,
        left=0.05,
        right=0.99,
        top=0.99,
        width_ratios=[2.5, 1],
        hspace=0.05,
        wspace=0.08,
    ),
    nrows=3,
    ncols=2,
)
hide_toolbar(fig.canvas)
ax_1d, ax_argand, ax_2d, ax_empty1, ax_2d_bw, ax_empty2 = axes.ravel()
ax_empty1.axis("off")
ax_empty2.axis("off")
for ax in axes.ravel():
    ax.set_yticks([])
ax_argand.set_xticks([])
ax_1d.axes.get_xaxis().set_visible(False)
ax_2d.axes.get_xaxis().set_visible(False)
ax_1d.sharex(ax_2d_bw)
ax_2d.sharex(ax_2d_bw)
ax_2d_bw.set_xlabel("Re $m$")
for ax in (ax_2d, ax_2d_bw):
    ax.set_ylabel("Im $m$")
ax_1d.set_ylabel("$|A|^2$")
ax_argand.set_xlabel("Re $A$")
ax_argand.set_ylabel("Im $A$")
for ax in (ax_1d, ax_argand):
    ax.spines["left"].set_position("zero")
    ax.spines["bottom"].set_position("zero")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

for ax in (ax_2d, ax_2d_bw):
    ax.axhline(0, linewidth=0.5, c="black", linestyle="dotted")
mass_line_style = dict(alpha=0.3, color="red")

# 3D plot
pole_indicators = []
pole_indicators_bw = []
lines_1d = None
meshes = None


def plot3(*, z_cutoff, complex_rendering, **kwargs):
    global lines_1d, meshes

    z_bw = np_bw(plot_domain, **kwargs)
    z_f_vector = np_f_vector(plot_domain, **kwargs)
    if lines_1d is None:
        lines_1d = [
            ax_1d.plot(plot_domain, np.abs(z_f_vector) ** 2, label="$P$-vector")[0],
            ax_1d.plot(plot_domain, np.abs(z_bw) ** 2, label="Breit–Wigner")[0],
            ax_1d.axvline(kwargs["m1"], **mass_line_style),
            ax_1d.axvline(kwargs["m2"], **mass_line_style),
            ax_argand.scatter(z_f_vector.real, z_f_vector.imag, s=1),
            ax_argand.scatter(z_bw.real, z_bw.imag, s=1),
        ]
    else:
        lines_1d[0].set_ydata(np.abs(z_f_vector) ** 2)
        lines_1d[1].set_ydata(np.abs(z_bw) ** 2)
        lines_1d[2].set_xdata([kwargs["m1"]])
        lines_1d[3].set_xdata([kwargs["m2"]])
        lines_1d[4].set_offsets(np.stack([z_f_vector.real, z_f_vector.imag]).T)
        lines_1d[5].set_offsets(np.stack([z_bw.real, z_bw.imag]).T)
    ax_1d.relim()
    ax_1d.autoscale_view()
    ax_argand.ignore_existing_data_limits = True
    box = Bbox.union([
        lines_1d[4].get_datalim(ax_argand.transData),
        lines_1d[5].get_datalim(ax_argand.transData),
    ])
    ax_argand.update_datalim(box)
    ax_argand.autoscale_view()

    Z = np_f_vector(plot_domain_complex, **kwargs)
    Z_bw = np_bw(plot_domain_complex, **kwargs)
    if complex_rendering == "imag":
        projection = np.imag
        ax_title = "Im $A$"
    elif complex_rendering == "real":
        projection = np.real
        ax_title = "Re $A$"
    elif complex_rendering == "abs":
        projection = np.vectorize(lambda z: np.abs(z) ** 2)
        ax_title = "$|A|$"
    else:
        raise NotImplementedError
    ax_2d.set_title(ax_title + " ($P$-vector)", y=0.87)
    ax_2d_bw.set_title(ax_title + " (Breit–Wigner)", y=0.87)

    if meshes is None:
        meshes = [
            ax_2d.pcolormesh(X, Y, projection(Z), cmap="RdBu"),
            ax_2d_bw.pcolormesh(X, Y, projection(Z_bw), cmap="RdBu"),
        ]
    else:
        meshes[0].set_array(projection(Z))
        meshes[1].set_array(projection(Z_bw))
    for mesh in meshes:
        mesh.set_clim(vmin=-z_cutoff, vmax=+z_cutoff)

    if pole_indicators:
        for R, (line, text) in enumerate(pole_indicators, 1):
            mass = kwargs[f"m{R}"]
            line.set_xdata([mass])
            text.set_x(mass + (x_max - x_min) * 0.008)
    else:
        for R in (1, 2):
            mass = kwargs[f"m{R}"]
            line = ax_2d.axvline(mass, **mass_line_style)
            text = ax_2d.text(
                x=mass + (x_max - x_min) * 0.008,
                y=0.95 * y_min,
                s=f"$m_{R}$",
                c="red",
            )
            pole_indicators.append((line, text))

    if pole_indicators_bw:
        for R, (line, text) in enumerate(pole_indicators_bw, 1):
            mass = kwargs[f"m{R}"]
            line.set_xdata([mass])
            text.set_x(mass + (x_max - x_min) * 0.008)
    else:
        for R in range(1, 2 + 1):
            mass = kwargs[f"m{R}"]
            line = ax_2d_bw.axvline(mass, **mass_line_style)
            text = ax_2d_bw.text(
                x=mass + (x_max - x_min) * 0.008,
                y=0.95 * y_min,
                s=f"$m_{R}$",
                c="red",
            )
            pole_indicators_bw.append((line, text))


# Create switch for imag/real/abs
sliders["complex_rendering"] = w.RadioButtons(
    description=R"\(s\)-plane plot",
    options=["imag", "real", "abs"],
)

# Create cut-off slider for z-direction
sliders["z_cutoff"] = w.FloatSlider(
    description=R"\(z\)-cutoff",
    min=0,
    max=+5,
    step=0.1,
    value=2,
)

# Create GUI
sliders_copy = dict(sliders)
slider_groups = []
for R in range(1, 2 + 1):
    vertical_slider_group = [
        sliders_copy.pop(f"c{R}"),
        sliders_copy.pop(f"phi{R}"),
        sliders_copy.pop(f"m{R}"),
        sliders_copy.pop(f"Gamma{R}"),
        sliders_copy.pop(f"gamma{R}"),
    ]
    slider_groups.append(vertical_slider_group)
slider_pairs = np.array(slider_groups).T
h_boxes = [w.HBox(tuple(pair)) for pair in slider_pairs]
remaining_sliders = sorted(
    sliders_copy.values(),
    key=lambda s: (str(type(s)), s.description),
)
ui = w.VBox(h_boxes + remaining_sliders)
output = w.interactive_output(plot3, controls=sliders)
ax_1d.legend(loc="upper right")
display(ui, output)

In [ ]:
if STATIC_WEB_PAGE:
    output_path = "p-vector-comparison.png"
    plt.savefig(output_path)
    display(Image(output_path))

## Interactive visualization

All $\boldsymbol{K}$-matrices can be inspected interactively for arbitrary poles and channels with the following applet:

[^pole-vs-resonance]: See {pdg-review}`2021; Resonances`, Section 50.1, for a discussion about what poles and resonances are. See also the intro to Section 5 in {cite}`Chung:1995dx`.

In [ ]:
if STATIC_WEB_PAGE:
    L = 0


def plot(
    kmatrix_type: kmatrix.TMatrix,
    n_channels: int,
    n_poles: int,
    angular_momentum=0,
    phsp_factor=phasespace.PhaseSpaceFactor,
    substitute_sqrt_rho: bool = False,
) -> None:
    i, j = sp.symbols("i, j", integer=True, negative=False)
    j = i
    expr = kmatrix_type.formulate(
        n_poles=n_poles,
        n_channels=n_channels,
        angular_momentum=angular_momentum,
        phsp_factor=phsp_factor,
    ).doit()[i, j]
    expr = substitute_indexed_symbols(expr)
    if substitute_sqrt_rho:

        def rho_i(i):
            return phsp_factor(*sp.symbols(f"s m_a{i} m_b{i}", nonnegative=True)).doit()

        expr = expr.xreplace({
            sp.sqrt(rho_i(i)): 1 for i in range(n_channels)
        }).xreplace({sp.conjugate(sp.sqrt(rho_i(i))): 1 for i in range(n_channels)})
    expr = expr.xreplace({s: m**2})
    expr = substitute_indexed_symbols(expr)
    args = [m, *sorted(expr.free_symbols - {m}, key=str)]
    func = sp.lambdify(args, expr)
    to_args = {
        str(s): arg
        for s, arg in zip(args, inspect.signature(func).parameters, strict=True)
    }
    sliders = {str(s): create_slider(s, max=1, step=0.01) for s in args[1:]}

    # Set plot domain
    x_min, x_max = 1e-3, 3
    y_min, y_max = -0.5, +0.5

    x = np.linspace(x_min, x_max, num=500) + 1e-8j
    X, Y = np.meshgrid(
        np.linspace(x_min, x_max, num=160),
        np.linspace(y_min, y_max, num=80),
    )
    XY = X + Y * 1j

    # Set slider values and ranges
    m0_values = np.linspace(x_min, x_max, num=n_poles + 2)
    m0_values = m0_values[1:-1]
    if "L" in sliders:
        sliders["L"].min = 0
        sliders["L"].max = 10
    sliders["i"].min = 0
    sliders["i"].max = n_channels - 1
    gamma_default = 1 / np.sqrt(n_channels * n_poles)
    for R in range(1, n_poles + 1):
        for i in range(n_channels):
            if kmatrix_type in {
                kmatrix.RelativisticKMatrix,
                kmatrix.RelativisticPVector,
            }:
                sliders[f"m{R}"].max = 3
                sliders[f"m{R}"].value = m0_values[R - 1]
                sliders[f"m_a{i}"].value = (i + 1) * 0.25
                sliders[f"m_b{i}"].value = (i + 1) * 0.25
                sliders[Rf"\Gamma_{{{R},{i}}}"].value = 0.35 + R * 0.2 - i * 0.3
                sliders[Rf"\Gamma_{{{R},{i}}}"].min = -1.5
                sliders[Rf"\Gamma_{{{R},{i}}}"].max = +1.5
                sliders[Rf"\gamma_{{{R},{i}}}"].min = -1
                sliders[Rf"\gamma_{{{R},{i}}}"].value = gamma_default
            if kmatrix_type in {
                kmatrix.NonRelativisticKMatrix,
                kmatrix.NonRelativisticPVector,
            }:
                sliders[f"m{R}"].max = 3
                sliders[f"m{R}"] = m0_values[R - 1]
                sliders[Rf"\Gamma_{{{R},{i}}}"].min = -1
                sliders[Rf"\gamma_{{{R},{i}}}"].min = -1
                sliders[Rf"\Gamma_{{{R},{i}}}"].value = (R + 1) * 0.1
                sliders[Rf"\gamma_{{{R},{i}}}"].value = gamma_default
            if kmatrix_type in {
                kmatrix.NonRelativisticPVector,
                kmatrix.RelativisticPVector,
            }:
                sliders[f"beta{R}"].min = -1
                sliders[f"beta{R}"].value = 1

    # Create interactive plots
    fig, axes = plt.subplots(figsize=(8, 6), nrows=2, sharex=True)
    fig.subplots_adjust(bottom=0.09, left=0.08, hspace=0.06, right=0.99, top=0.9)
    hide_toolbar(fig.canvas)
    if kmatrix_type in {
        kmatrix.NonRelativisticKMatrix,
        kmatrix.RelativisticKMatrix,
    }:
        suptitle = (
            Rf"${n_channels} \times {n_channels}$ $K$-matrix with {n_poles} resonances"
        )
    elif kmatrix_type in {
        kmatrix.NonRelativisticPVector,
        kmatrix.RelativisticPVector,
    }:
        suptitle = f"$P$-vector for {n_channels} channels and {n_poles} resonances"
    fig.suptitle(suptitle, y=0.94)

    for ax in axes:
        ax.set_xlim(x_min, x_max)
    ax1, ax2 = axes.ravel()
    ax1.set_ylabel("$|T|^{2}$")
    ax1.set_yticks([])
    ax2.set_xlabel("Re $m$")
    ax2.set_ylabel("Im $m$")

    ax2.axhline(0, linewidth=0.5, c="black", linestyle="dotted")

    mass_line_style = dict(alpha=0.3, color="red")
    lines_2d = None
    mesh = None
    pole_indicators = []
    threshold_indicators = []

    def plot(*, z_cutoff, complex_rendering, **kwargs):
        nonlocal lines_2d, mesh
        func_kwargs = {to_args[k]: v for k, v in kwargs.items() if k in to_args}
        Z = func(XY, **func_kwargs)
        if complex_rendering == "imag":
            Z = Z.imag
            ax_title = "Im $T$"
        elif complex_rendering == "real":
            Z = Z.real
            ax_title = "Re $T$"
        elif complex_rendering == "abs":
            Z = np.abs(Z)
            ax_title = "$|T|$"
        else:
            raise NotImplementedError

        if n_channels == 1:
            ax2.set_title(ax_title, y=0.87)
        else:
            i = kwargs["i"]
            ax2.set_title(f"{ax_title}, channel {i}", y=0.87)

        y = np.abs(func(x, **func_kwargs)) ** 2
        if lines_2d is None:
            lines_2d = [
                ax1.plot(x, y, label=f"channel {i}")[0] for i in range(n_channels)
            ]
        else:
            for line in lines_2d:
                line.set_ydata(y)

        if mesh is None:
            mesh = ax2.pcolormesh(X, Y, Z, cmap="RdBu")
        else:
            mesh.set_array(Z)
        mesh.set_clim(vmin=-z_cutoff, vmax=+z_cutoff)

        if pole_indicators:
            for R, (line, text) in enumerate(pole_indicators, 1):
                mass = kwargs[f"m{R}"]
                line.set_xdata([mass])
                text.set_x(mass + (x_max - x_min) * 0.008)
        else:
            for R in range(1, n_poles + 1):
                mass = kwargs[f"m{R}"]
                line = ax2.axvline(mass, **mass_line_style)
                text = ax2.text(
                    x=mass + (x_max - x_min) * 0.008,
                    y=0.95 * y_min,
                    s=f"$m_{R}$",
                    c="red",
                )
                pole_indicators.append((line, text))

        if kmatrix_type is kmatrix.RelativisticKMatrix:
            x_offset = (x_max - x_min) * 0.015
            if threshold_indicators:
                for i, (line_thr, line_diff, text_thr, text_diff) in enumerate(
                    threshold_indicators
                ):
                    m_a = kwargs[f"m_a{i}"]
                    m_b = kwargs[f"m_b{i}"]
                    s_thr = m_a + m_b
                    m_diff = abs(m_a - m_b)
                    line_thr.set_xdata([s_thr])
                    line_diff.set_xdata([m_diff])
                    text_thr.set_x(s_thr)
                    text_diff.set_x(m_diff - x_offset)
            else:
                colors = cm.plasma(np.linspace(0, 1, n_channels))
                for i, color in enumerate(colors):
                    m_a = kwargs[f"m_a{i}"]
                    m_b = kwargs[f"m_b{i}"]
                    s_thr = m_a + m_b
                    m_diff = abs(m_a - m_b)
                    line_thr = ax.axvline(s_thr, c=color, linestyle="dotted")
                    line_diff = ax.axvline(m_diff, c=color, linestyle="dashed")
                    text_thr = ax.text(
                        x=s_thr,
                        y=0.95 * y_min,
                        s=f"$m_{{a{i}}}+m_{{b{i}}}$",
                        c=color,
                        rotation=-90,
                    )
                    text_diff = ax.text(
                        x=m_diff - x_offset,
                        y=0.95 * y_min,
                        s=f"$m_{{a{i}}}-m_{{b{i}}}$",
                        c=color,
                        rotation=+90,
                    )
                    threshold_indicators.append((
                        line_thr,
                        line_diff,
                        text_thr,
                        text_diff,
                    ))
            for i, (_, line_diff, _, text_diff) in enumerate(threshold_indicators):
                m_a = kwargs[f"m_a{i}"]
                m_b = kwargs[f"m_b{i}"]
                s_thr = m_a + m_b
                m_diff = abs(m_a - m_b)
                if m_diff > x_offset + 0.01 and s_thr - abs(m_diff) > x_offset:
                    line_diff.set_alpha(0.5)
                    text_diff.set_alpha(0.5)
                else:
                    line_diff.set_alpha(0)
                    text_diff.set_alpha(0)

    # Create switch for imag/real/abs
    sliders["complex_rendering"] = w.RadioButtons(
        description=R"\(s\)-plane plot",
        options=["imag", "real", "abs"],
    )

    # Create cut-off slider for z-direction
    sliders["z_cutoff"] = w.FloatSlider(
        description=R"\(z\)-cutoff",
        min=+0.01,
        max=+5,
        step=0.01,
        value=1,
    )

    # Link sliders
    if kmatrix_type is kmatrix.RelativisticKMatrix:
        for i in range(n_channels):
            w.dlink(
                (sliders[f"m_a{i}"], "value"),
                (sliders[f"m_b{i}"], "value"),
            )

    # Create GUI
    sliders_copy = dict(sliders)
    h_boxes = []
    for R in range(1, n_poles + 1):
        buttons = [sliders_copy.pop(f"m{R}")]
        if n_channels == 1:
            buttons += [
                sliders_copy.pop(Rf"\Gamma_{{{R},0}}"),
                sliders_copy.pop(Rf"\gamma_{{{R},0}}"),
            ]
        h_box = w.HBox(buttons)
        h_boxes.append(h_box)
    remaining_sliders = sorted(
        sliders_copy.values(),
        key=lambda s: (str(type(s)), s.description),
    )
    if n_channels == 1:
        remaining_sliders.remove(sliders["i"])
    ui = w.VBox(h_boxes + remaining_sliders)
    output = w.interactive_output(plot, controls=sliders)
    if n_channels > 1:
        ax1.legend(loc="upper right")
    display(ui, output)

{{ run_interactive }}

In [ ]:
plot(
    kmatrix.RelativisticKMatrix,
    n_poles=2,
    n_channels=1,
    angular_momentum=0,
    phsp_factor=phasespace.PhaseSpaceFactor,
    substitute_sqrt_rho=False,
)

In [ ]:
if STATIC_WEB_PAGE:
    output_path = "k-matrix.png"
    plt.savefig(output_path, dpi=150)
    display(Image(output_path))